# Spis treści
- [O Guidance](../../../../code/01.Introduce)
- [Konfiguracja](../../../../code/01.Introduce)
- [Generowanie bez ograniczeń](../../../../code/01.Introduce)
- [Mówienie za Phi 3](../../../../code/01.Introduce)
- [Regex](../../../../code/01.Introduce)
- [Wybór](../../../../code/01.Introduce)
- [Łańcuch myśli](../../../../code/01.Introduce)
- [Generowanie JSON](../../../../code/01.Introduce)
- [Generowanie HTML](../../../../code/01.Introduce)


# O Guidance

Guidance to sprawdzona, otwartoźródłowa biblioteka Python służąca do kontrolowania wyników dowolnego modelu językowego (LM). Za pomocą jednego wywołania API możesz wyrazić (w Pythonie) precyzyjne programowe ograniczenie(-a), które model musi przestrzegać, i wygenerować uporządkowany wynik w formacie JSON, Python, HTML, SQL lub w dowolnej strukturze wymaganej przez dany przypadek użycia.

Guidance różni się od tradycyjnych technik podpowiadania. Wymusza ograniczenia, kierując modelem token po tokenie na warstwie inferencji, co pozwala uzyskać wyższej jakości wyniki oraz zmniejszyć koszty i opóźnienia nawet o 30–50% w przypadku scenariuszy wymagających wysokiej struktury.

Aby dowiedzieć się więcej o Guidance, odwiedź [publiczne repozytorium na GitHub](https://github.com/guidance-ai/guidance) lub obejrzyj [sesję Guidance Breakout](https://www.youtube.com/watch?v=qXMNPVVlCMs) podczas Microsoft Build.


# Konfiguracja
1. Zainstaluj Guidance za pomocą `pip install guidance --pre`
2. Wdróż punkt końcowy Phi 3.5 mini w Azure, przechodząc na stronę https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml i klikając przycisk "Deploy"
3. Zapisz klucz API swojego punktu końcowego w zmiennej środowiskowej o nazwie `AZURE_PHI3_KEY` oraz URL w zmiennej środowiskowej o nazwie `AZURE_PHI3_URL`


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# Generowanie bez ograniczeń
Tekst można generować bez żadnych ograniczeń za pomocą funkcji `gen()`. Jest to takie samo jak korzystanie z modelu bez Guidance.

## Formatowanie rozmowy
Podobnie jak wiele modeli czatu, Phi-3 oczekuje wiadomości między użytkownikiem a asystentem w określonym formacie. Guidance obsługuje szablon czatu Phi-3 i zajmie się formatowaniem rozmowy za Ciebie. Aby stworzyć wymiany w rozmowie, umieść każdą część konwersacji w bloku `with user()` lub `with assistant()`. Blok `with system()` może być używany do ustawienia wiadomości systemowej.


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## Oszczędność tokenów
W mocno ustrukturyzowanych scenariuszach Guidance może pomijać tokeny i generować tylko te niezbędne, co poprawia wydajność, zwiększa efektywność i obniża koszty API. Wygenerowane tokeny są pokazane w tym notatniku z wyróżnionym tłem. Wymuszone tokeny są pokazane bez wyróżnienia i kosztują tyle samo co tokeny wejściowe, których koszt szacuje się na jedną trzecią kosztu tokenów wyjściowych.

*Uwaga:* Pierwszy przykład z nieograniczoną generacją nie był w stanie wymusić żadnych tokenów, ponieważ nie zastosowaliśmy żadnych ograniczeń.


# Mówienie za Phi 3
Dzięki Guidance możesz łatwo wprowadzać tekst do odpowiedzi modelu. Może to być przydatne, jeśli chcesz ukierunkować odpowiedź modelu w określonym kierunku.


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# Ograniczanie za pomocą wyrażeń regularnych
W poprzednim przykładzie Phi 3 odpowiedział dodatkowymi wyjaśnieniami po udzieleniu odpowiedzi `Canberra`. Aby ograniczyć odpowiedź modelu do dokładnie jednego słowa, można użyć wyrażenia regularnego.


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

Za pomocą wyrażenia regularnego generowane jest tylko słowo `Canberra`.


# Wybór spośród wielu opcji
Kiedy znane są niektóre możliwe opcje, możesz użyć funkcji `select()`, aby model wybrał z listy dostępnych możliwości.


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

Za pomocą `select()` wygenerowano tylko token `Can`. Ponieważ `Canberra` jest jedyną opcją, która może uzupełnić odpowiedź, pozostałe tokeny zostały wymuszone.


# Łańcuch Myśli
Łańcuch myśli to technika, która może pomóc poprawić jakość wyników generowanych przez model, zachęcając go do rozwiązywania problemu krok po kroku. Zazwyczaj, aby dojść do ostatecznej odpowiedzi, konieczne są wielokrotne iteracje w ramach promptów. Najpierw należy poinstruować model, aby myślał etapami. Następnie ponownie poprosić model o podanie ostatecznej odpowiedzi. W przypadku standardowych API do wnioskowania w rozmowach, wymaga to dwóch wywołań API, a wygenerowany przez model „łańcuch myśli” jest naliczany dwukrotnie – raz jako tokeny wyjściowe, gdy model go wygenerował, a potem ponownie jako tokeny wejściowe podczas drugiego wywołania. Dzięki Guidance, cały proces wieloetapowy jest przetwarzany i naliczany w ramach jednego wywołania API, co zmniejsza koszty i opóźnienia.


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# Generowanie JSON
Guidance może być używane do zapewnienia generowania JSON zgodnego ze schematem JSON lub modelem pydantic, takim jak pokazany tutaj schemat profilu użytkownika.


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## Generowanie HTML

Guidance może być również używane do generowania kodu i przestrzegania wymagań składniowych w języku programowania. W tej sekcji stworzymy mały program Guidance do pisania bardzo prostych stron internetowych w HTML.

Podzielimy stronę internetową na mniejsze sekcje, z których każda będzie miała swoją własną funkcję Guidance. Następnie połączymy je w naszej ostatecznej funkcji, aby stworzyć stronę internetową w HTML.
Na koniec uruchomimy tę funkcję na modelu obsługującym Guidance w Azure AI.

*Uwaga:* Nie będzie to w pełni funkcjonalny generator HTML; celem jest pokazanie, jak można tworzyć uporządkowane dane wyjściowe dostosowane do indywidualnych potrzeb.

Zaczynamy od zaimportowania tego, czego potrzebujemy z Guidance:


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

Strony internetowe w formacie HTML są bardzo uporządkowane, a my będziemy „wymuszać” te części strony za pomocą Guidance.  
Kiedy wyraźnie wymagamy tekstu od modelu, musimy upewnić się, że nie zawiera on niczego, co mogłoby być traktowane jako znacznik - to znaczy, musimy wykluczyć znaki '<' i '>'.


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

Możemy następnie użyć tej funkcji do generowania tekstu w dowolnym znaczniku HTML:


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

Teraz utwórzmy nagłówek strony.  
W ramach tego musimy wygenerować tytuł strony:


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

Treść strony HTML będzie wypełniona nagłówkami i akapitami.  
Możemy zdefiniować funkcję, która zajmie się każdym z tych elementów:


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

Teraz funkcja definiująca treść samego HTML-a.  
Wykorzystuje `select()` z `recurse=True`, aby wygenerować wiele nagłówków i akapitów:


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

Następnie przechodzimy do funkcji, która generuje kompletną stronę HTML.  
Dodajemy początkowy tag HTML, potem generujemy nagłówek, następnie treść, a na końcu dołączamy końcowy tag HTML:


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

Oferujemy przyjazny dla użytkownika wrapper, który pozwoli nam:
- Ustawić temperaturę generowania
- Przechwycić wygenerowaną stronę z obiektu Model


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

Możemy następnie zapisać wynik do pliku:


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

I zobacz [wynik](../../../../code/01.Introduce/sample_page.html).



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczeniowej AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było precyzyjne, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za źródło autorytatywne. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
